# Mikroskil Data Science Assignment Helper Notebook

> **Read me first:** This notebook is a teaching aid. It shows how to plan work that satisfies the rubric and scenarios without applying the steps to the confidential dataset. Replace every placeholder with your own implementation, document your choices, and do not submit this file as-is.

## How to use this template

- Skim the rubric summary to build your personal checklist.
- Use the Phase 1 and Phase 2 scaffolding to outline your own notebook.
- Each scenario playbook maps the official questions to analysis patterns; adapt them with real data.
- Replace every `raise NotImplementedError` block with working code once you understand the intent.
- Keep a decision log in the provided cell to document cleaning rules and assumptions.

## Rubric at a glance

### Phase 1: Python Structures, Gathering & Cleaning
| Criterion | Evidence to produce | Weight |
| --- | --- | --- |
| Dataset understanding | Load CSV, inspect `head()`, `info()`, and explain meaning of key columns. | 10% |
| Raw Python data structures | Lists, tuples, dicts built from a manageable subset (first 5-10 rows). | 20% |
| Numpy & pandas structures | Use `np.ndarray`, `pd.Series`, clean base `DataFrame` with correct dtypes. | 15% |
| Indexing & subsetting | Demonstrate `.loc`, `.iloc`, boolean filters, aggregations tied to scenario. | 15% |
| Data gathering / reshaping | Group, pivot, merge, create derived columns fit for scenario. | 15% |
| Data cleaning | Handle types, NaN, outliers, useless columns; justify every choice. | 20% |
| Code quality & documentation | Modular, readable code, comments, clear layout. | 5% |

### Phase 2: EDA, Visualization, Analysis
| Criterion | Evidence to produce | Weight |
| --- | --- | --- |
| EDA breadth & depth | Scenario-relevant descriptive stats, feature engineering, comparative views. | 20% |
| Visualization quality | 3-5 well-chosen plots with titles, labels, readable legends. | 25% |
| Scenario-specific metrics | Compute and interpret the required KPIs for your scenario. | 25% |
| Interpretation & insights | Write narrative conclusions, highlight limitations. | 20% |
| Communication & storytelling | Structured notebook: intro → method → results → recommendations. | 10% |

## Suggested workflow

1. Read the scenario brief carefully and highlight required metrics.
2. Sketch deliverables for Phase 1 and Phase 2 in your own words.
3. Duplicate this helper into a working notebook and remove placeholders.
4. Load the dataset, build raw Python structures, then transition to pandas.
5. Tackle cleaning tasks before deep analysis; log every decision.
6. Build EDA visuals, answer scenario questions, and draft insights.
7. Review rubric weights and verify that each item has explicit evidence.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configure pandas display for wide tables; adjust as needed.
pd.options.display.max_columns = 100

DATA_PATH = Path("../claim_invoice_items_data.csv")
if not DATA_PATH.exists():
    raise FileNotFoundError("Update DATA_PATH so it points at the invoice-item CSV before running this helper.")

### Data loading helper

Replace the placeholder path above and wire up the loader function below. Add `dtype` hints if the CSV has scientific notation or numeric strings.

In [1]:
def load_invoice_items(path: Path) -> pd.DataFrame:
    """Read the raw invoice-item CSV into a pandas DataFrame."""
    # TODO: adjust dtype arguments or parse_dates once you inspect the CSV header.
    df = pd.read_csv(
        path,
        parse_dates=["created_at", "updated_at", "deleted_at", "indate", "outdate"],
    )
    return df

# Uncomment after configuring DATA_PATH
raw_df = load_invoice_items(DATA_PATH)
raw_df.head()

,id,result_id,created_at,updated_at,deleted_at,provider,provider_code,claim_id,benefit_category,benefit_name,category,chemical_substance,description,item_label,item_type,kelas,matched_name,nie_bpom,quantity,sub_benefit_name,tarif_description,tarif_price,total_price,unit,unit_price,invoice_number,indate,outdate,card_number,member_name,sex,dob,corporate_name,icd10_1,description_1,icd10_2,icd10_3,icd10_4,provider_name,province,country
0,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,B05BA01 - Amino acids,Aminoped Infusion 100 mL,charge_item,invoice,NaN,"Aminofusin L 600 Infus (AMINOFUSIN L 600, 500 mL)",DKL0411637149A1,1.0,Obat-Obatan,-,NaN,1.209075,NaN,1.751214,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
1,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,N02BE01 - Paracetamol,Intrizin Drops 15 mL,charge_item,invoice,NaN,"Paracetamol 100 mg/mL Drops (NIPE FEVER, 15 mL)",DBL2032209736A1,1.0,Obat-Obatan,-,NaN,2.927870,NaN,1.559603,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
2,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,A02BC01 - Omeprazole,Pumpitor Injeksi 40 mg,charge_item,invoice,NaN,Omeprazole Sodium 40 mg Serbuk Injeksi (PUMPITOR),DKL1022248644A1,1.0,Obat-Obatan,-,NaN,5.753584,NaN,5.111212,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
3,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,N02BE01 - Paracetamol,Praxion Drops 15 mL,charge_item,invoice,NaN,Paracetamol 100 mg/mL Drops (PRAXION),DBL0521631536A1,1.0,Obat-Obatan,-,NaN,1.133660,NaN,0.839043,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
4,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,R03CC02 - Salbutamol,Velutine Nebules 2.5 mg/4 mL,charge_item,invoice,NaN,"Salbutamol Sulfate 2,5 mg/4 mL Cairan Inhalasi...",DKL1933529168B1,1.0,Obat-Obatan,-,NaN,0.118520,NaN,0.217683,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia


### Decision log helper

Document every material assumption, cleaning rule, or exclusion so the instructor can follow your reasoning.

In [6]:
decision_log = []

def log_decision(step: str, choice: str, impact: str) -> None:
    """Append a structured note to the decision log."""
    note = {"step": step, "choice": choice, "impact": impact}
    decision_log.append(note)

# Example placeholder (delete after adding your own entries)
log_decision("template", "Describe your first decision here", "Summarize the effect on the analysis")
decision_log

[{'step': 'template',
  'choice': 'Describe your first decision here',
  'impact': 'Summarize the effect on the analysis'}]

## Phase 1: Data structures, gathering, and cleaning

### 1. Dataset understanding (10%)

- Load the CSV, inspect `head()`, `info()`, and `describe(include="all")`.
- In a markdown cell, explain what each key column (provider, benefit_category, quantity, total_price, icd10_1, etc.) represents.
- Clarify the unit of analysis (invoice item) and the meaning of rows vs columns.

In [ ]:
def summarize_dataset(df: pd.DataFrame) -> None:
    """Template for reporting dataset shape and basic column meanings."""
    if df is None:
        raise ValueError("Pass the loaded DataFrame before calling this helper.")
    print(f"Rows: {df.shape[0]:,}  Columns: {df.shape[1]}")
    display(df.head(3))
    display(df.info())
    display(df.describe(include="all"))
    # TODO: add narrative summary in a markdown cell after running this function.

summarize_dataset(raw_df)  # Uncomment once raw_df is defined


"This answers the first question on the scenario pdf."
def list_numeric_columns_and_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    1) List numeric columns and compute min, max, mean.

    Returns a DataFrame indexed by column name with columns: dtype, min, max, mean, non_null_count.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # TODO add handling for numeric-like object columns. dob is date of birth, so not numeric. consider later.
    # In some datasets numeric-like columns are stored as object; consider those later, but
    # this function strictly lists columns already numeric.
    stats = []
    for col in numeric_cols:
        col_ser = df[col].dropna()
        stats.append(
            {
                "column": col,
                "dtype": df[col].dtype,
                "min": col_ser.min() if not col_ser.empty else np.nan,
                "max": col_ser.max() if not col_ser.empty else np.nan,
                "mean": col_ser.mean() if not col_ser.empty else np.nan,
                "non_null_count": col_ser.shape[0],
            }
        )
    stats_df = pd.DataFrame(stats).set_index("column")
    return stats_df

list_numeric_columns_and_stats(raw_df)

Rows: 264,694  Columns: 41


,id,result_id,created_at,updated_at,deleted_at,provider,provider_code,claim_id,benefit_category,benefit_name,category,chemical_substance,description,item_label,item_type,kelas,matched_name,nie_bpom,quantity,sub_benefit_name,tarif_description,tarif_price,total_price,unit,unit_price,invoice_number,indate,outdate,card_number,member_name,sex,dob,corporate_name,icd10_1,description_1,icd10_2,icd10_3,icd10_4,provider_name,province,country
0,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,B05BA01 - Amino acids,Aminoped Infusion 100 mL,charge_item,invoice,NaN,"Aminofusin L 600 Infus (AMINOFUSIN L 600, 500 mL)",DKL0411637149A1,1.0,Obat-Obatan,-,NaN,1.209075,NaN,1.751214,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
1,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,N02BE01 - Paracetamol,Intrizin Drops 15 mL,charge_item,invoice,NaN,"Paracetamol 100 mg/mL Drops (NIPE FEVER, 15 mL)",DBL2032209736A1,1.0,Obat-Obatan,-,NaN,2.927870,NaN,1.559603,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia
2,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848 UTC,2025-10-26 16:08:32.848 UTC,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,A02BC01 - Omeprazole,Pumpitor Injeksi 40 mg,charge_item,invoice,NaN,Omeprazole Sodium 40 mg Serbuk Injeksi (PUMPITOR),DKL1022248644A1,1.0,Obat-Obatan,-,NaN,5.753584,NaN,5.111212,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264694 entries, 0 to 264693
Data columns (total 41 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  264694 non-null  object        
 1   result_id           264694 non-null  object        
 2   created_at          264694 non-null  object        
 3   updated_at          264694 non-null  object        
 4   deleted_at          0 non-null       datetime64[ns]
 5   provider            264694 non-null  object        
 6   provider_code       264694 non-null  object        
 7   claim_id            264694 non-null  object        
 8   benefit_category    262162 non-null  object        
 9   benefit_name        264694 non-null  object        
 10  category            254764 non-null  object        
 11  chemical_substance  262162 non-null  object        
 12  description         262475 non-null  object        
 13  item_label          247410 no

None

,id,result_id,created_at,updated_at,deleted_at,provider,provider_code,claim_id,benefit_category,benefit_name,category,chemical_substance,description,item_label,item_type,kelas,matched_name,nie_bpom,quantity,sub_benefit_name,tarif_description,tarif_price,total_price,unit,unit_price,invoice_number,indate,outdate,card_number,member_name,sex,dob,corporate_name,icd10_1,description_1,icd10_2,icd10_3,icd10_4,provider_name,province,country
count,264694,264694,264694,264694,0,264694,264694,264694,262162,264694,254764,262162,262475,247410,262162,48265,262162,262162,264691.000000,264694,262150,88673.000000,2.439330e+05,105863,2.646940e+05,264694,264694,264694,264694,264694,264694,264694.000000,264694,264694,264694,80178,30485,11740,264694,264694,264694
unique,13734,13734,13734,13734,NaN,65,64,13734,26,7,80,824,41839,18,1,74,10240,3500,NaN,56,6240,NaN,NaN,799,NaN,2357,NaN,NaN,8675,8412,4,NaN,79,1360,1333,457,292,186,65,33,11
top,08655050-f04e-4d9d-b18a-427bc58a6ad2,701f0ca9-ed82-4718-a0d0-c72d1f5fb7dc,2025-11-05 01:15:25.756 UTC,2025-11-05 01:15:25.756 UTC,NaN,PROVIDER_000056,PROVCODE_000053,CLAIMID_004108,medicine,RAWAT INAP,Obat-obatan,-,Konsultasi Dokter Umum,charge_item,invoice,Rawat Jalan TK 1,Not Found,-,NaN,Others,-,NaN,NaN,unspecify,NaN,INVOICENO_000357,NaN,NaN,CARD_008664,MEMBER_006104,F,NaN,CORP_000051,R50.9,"Fever, unspecified",R50.9,E78.5,K30,PROVNAME_000056,Da Nang,Vietnam
freq,1332,1332,1332,1332,NaN,29895,29895,1332,88222,115767,56463,192911,9479,147407,262162,23340,100455,194116,NaN,82102,174016,NaN,NaN,27112,NaN,11488,NaN,NaN,1332,1332,146792,NaN,24124,14139,14249,6417,2700,1157,29895,64139,85230
mean,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.107707,NaN,NaN,20.820751,1.826217e+09,NaN,1.190605e+08,NaN,2025-10-02 03:58:29.691946240,2025-10-04 01:11:03.954604288,NaN,NaN,NaN,1983.018826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-39.000000,NaN,NaN,0.000000,-4.774980e+11,NaN,-3.872193e+03,NaN,2025-02-01 00:00:00,2025-02-01 00:00:00,NaN,NaN,NaN,1930.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.762859,0.000000e+00,NaN,0.000000e+00,NaN,2025-09-27 00:00:00,2025-09-29 00:00:00,NaN,NaN,NaN,1960.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,5.194091,4.436478e-01,NaN,3.059791e-01,NaN,2025-10-07 00:00:00,2025-10-09 00:00:00,NaN,NaN,NaN,1984.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,17.005917,2.392580e+00,NaN,1.579669e+00,NaN,2025-10-16 00:00:00,2025-10-18 00:00:00,NaN,NaN,NaN,2001.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30728.000000,NaN,NaN,3104.277906,4.885550e+13,NaN,1.072870e+12,NaN,2025-11-03 00:00:00,2025-11-03 00:00:00,NaN,NaN,NaN,2025.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,dtype,min,max,mean,non_null_count
column,,,,,
quantity,float64,-3.900000e+01,3.072800e+04,4.107707e+00,264691
tarif_price,float64,0.000000e+00,3.104278e+03,2.082075e+01,88673
total_price,float64,-4.774980e+11,4.885550e+13,1.826217e+09,243933
unit_price,float64,-3.872193e+03,1.072870e+12,1.190605e+08,264694
dob,int64,1.930000e+03,2.025000e+03,1.983019e+03,264694


## Data Overview
This dataset contains invoice-level healthcare billing data.
Each row represents a single invoice item, meaning one medical service, procedure, or medication billed under a particular provider’s invoice.
Multiple rows can belong to the same invoice if the patient received several services.

## Key Columns
- `id` Identifier of each item claims, can appear multiple times if claim has multiple items.
- `created_at` Timestamp when the item was captured in the source system.
- `claim_id` Claim header identifier grouping related invoice items.
- `item_label` Invoice line classification (charge, discount, cancel, placeholder)
- `item_type` Source line type.
- `matched_name` System data that matched with the line item on the invoice.
- `quantity` Quantity billed for the line item.
- `tarif_description` Nearest description for hospital charge tariffs item found in tarif reference database
- `tarif_price` Tariff amount tied to `tarif_description`.
- `total_price` Total billed amount for the item.

## Row and Column Representation
- Each row represents 1 invoice item (service or medicine product)
- Each column represents attribute of that item, such as `item_type`, `quantity`, `tarif_price`, `total_price`, etc.

### 2. Raw Python data structures (20%)

- Work with a small slice: `sample_records = df.head(10).to_dict(orient="records")`.
- Build lists (e.g., provider names), tuples (immutable identifiers), and dicts (e.g., code → description).
- Explain in text why each structure suits the scenario.

In [8]:
def build_python_structures(sample_records: list) -> dict:
    """Return example Python structures built from a small subset of rows."""
    if not sample_records:
        raise ValueError("Provide 5-10 records from your DataFrame.")

    providers = [row["provider_name"] for row in sample_records if "provider_name" in row]
    item_keys = [(row.get("id"), row.get("description")) for row in sample_records]
    provider_map = {row.get("provider_code"): row.get("provider_name") for row in sample_records}

    structures = {
        "providers_list": providers,
        "item_keys_tuple": item_keys[:3],
        "provider_lookup": provider_map,
    }
    return structures

example_structures = build_python_structures(raw_df.head(10).to_dict(orient="records"))
display(example_structures)

{'providers_list': ['PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048',
  'PROVNAME_000048'],
 'item_keys_tuple': [('85c3901c-cc38-4da2-b0e7-20dbf44623d0',
   'Aminoped Infusion 100 mL'),
  ('85c3901c-cc38-4da2-b0e7-20dbf44623d0', 'Intrizin Drops 15 mL'),
  ('85c3901c-cc38-4da2-b0e7-20dbf44623d0', 'Pumpitor Injeksi 40 mg')],
 'provider_lookup': {'PROVCODE_000031': 'PROVNAME_000048'}}

## Why each structure suits the screnario?
1. `provider_list` will help to determine which provider contributes the highest total claim cost?
2. `item_keys_tuple` will show which items or drug types contribute most to the claim cost
3. `provider_lookup` will simplifies grouping and joining for cost summaries

### 3. Numpy and pandas structures (15%)

- Convert relevant numeric columns to `np.ndarray` and `pd.Series`.
- Demonstrate vectorized operations (e.g., `df["unit_price"].to_numpy()` for calculations).
- After enforcing dtypes, create a clean working copy `clean_df = df.copy()` for downstream work.

In [9]:
def enforce_dtypes(df: pd.DataFrame, numeric_cols: list, date_cols: list) -> pd.DataFrame:
    """Create a cleaned copy with enforced numeric and datetime types."""
    clean_df = df.copy()
    for col in numeric_cols:
        clean_df[col] = pd.to_numeric(clean_df[col], errors="coerce")
    for col in date_cols:
        clean_df[col] = pd.to_datetime(clean_df[col], errors="coerce")

    # Vectorized numeric calculation
    clean_df["computed_total"] = clean_df["quantity"] * clean_df["unit_price"]

    # Vectorized date calculation
    clean_df["los_days"] = (clean_df["outdate"] - clean_df["indate"]).dt.days

    return clean_df

numeric_columns = ["quantity", "unit_price", "total_price"]
date_columns = ["created_at", "updated_at", "deleted_at", "indate", "outdate"]
clean_df = enforce_dtypes(raw_df, numeric_columns, date_columns)
clean_df.head()

,id,result_id,created_at,updated_at,deleted_at,provider,provider_code,claim_id,benefit_category,benefit_name,category,chemical_substance,description,item_label,item_type,kelas,matched_name,nie_bpom,quantity,sub_benefit_name,tarif_description,tarif_price,total_price,unit,unit_price,invoice_number,indate,outdate,card_number,member_name,sex,dob,corporate_name,icd10_1,description_1,icd10_2,icd10_3,icd10_4,provider_name,province,country,computed_total,los_days
0,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848000+00:00,2025-10-26 16:08:32.848000+00:00,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,B05BA01 - Amino acids,Aminoped Infusion 100 mL,charge_item,invoice,NaN,"Aminofusin L 600 Infus (AMINOFUSIN L 600, 500 mL)",DKL0411637149A1,1.0,Obat-Obatan,-,NaN,1.209075,NaN,1.751214,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia,1.751214,3
1,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848000+00:00,2025-10-26 16:08:32.848000+00:00,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,N02BE01 - Paracetamol,Intrizin Drops 15 mL,charge_item,invoice,NaN,"Paracetamol 100 mg/mL Drops (NIPE FEVER, 15 mL)",DBL2032209736A1,1.0,Obat-Obatan,-,NaN,2.927870,NaN,1.559603,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia,1.559603,3
2,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848000+00:00,2025-10-26 16:08:32.848000+00:00,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,A02BC01 - Omeprazole,Pumpitor Injeksi 40 mg,charge_item,invoice,NaN,Omeprazole Sodium 40 mg Serbuk Injeksi (PUMPITOR),DKL1022248644A1,1.0,Obat-Obatan,-,NaN,5.753584,NaN,5.111212,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia,5.111212,3
3,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848000+00:00,2025-10-26 16:08:32.848000+00:00,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,N02BE01 - Paracetamol,Praxion Drops 15 mL,charge_item,invoice,NaN,Paracetamol 100 mg/mL Drops (PRAXION),DBL0521631536A1,1.0,Obat-Obatan,-,NaN,1.133660,NaN,0.839043,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia,0.839043,3
4,85c3901c-cc38-4da2-b0e7-20dbf44623d0,1d8d7db7-ad0a-4123-a29a-9b100d48dbc6,2025-10-26 16:08:32.848000+00:00,2025-10-26 16:08:32.848000+00:00,NaT,PROVIDER_000048,PROVCODE_000031,CLAIMID_005005,medicine,RAWAT INAP,Obat-obatan,R03CC02 - Salbutamol,Velutine Nebules 2.5 mg/4 mL,charge_item,invoice,NaN,"Salbutamol Sulfate 2,5 mg/4 mL Cairan Inhalasi...",DKL1933529168B1,1.0,Obat-Obatan,-,NaN,0.118520,NaN,0.217683,INVOICENO_002045,2025-10-04,2025-10-07,CARD_008121,MEMBER_008336,F,2024,CORP_000026,J06.9,"Acute upper respiratory infection, unspecified",NaN,NaN,NaN,PROVNAME_000048,Central Java,Indonesia,0.217683,3


I converted numeric columns like quantity, unit_price, and total_price into both NumPy arrays and pandas Series to demonstrate vectorized computation.
NumPy arrays are efficient for numerical operations, while pandas Series retain column labels and indexes, making them easier to align with other data in the DataFrame.

### 4. Indexing and subsetting (15%)

- Prepare `.loc` and `.iloc` examples tied to your scenario filters.
- Demonstrate boolean filters (e.g., `clean_df[clean_df["benefit_category"] == "medicine"]`).
- Store at least one filtered subset for later EDA.

In [10]:
def demo_indexing(df: pd.DataFrame) -> None:
    """Showcase loc, iloc, and boolean indexing for rubric evidence."""
    # TODO: adapt column names and filters to your scenario.
    medicine_mask = df["benefit_category"].eq("medicine")
    sample_loc = df.loc[medicine_mask, ["provider_name", "quantity", "total_price"]].head()
    sample_iloc = df.iloc[:5, :5]
    display(sample_loc)
    display(sample_iloc)

# demo_indexing(clean_df)

### 5. Data gathering and reshaping (15%)

- Use `groupby`, `pivot_table`, or joins to engineer the structures your scenario needs.
- Create at least one derived column (e.g., `computed_total = quantity * unit_price`).
- If merging external mappings (ICD groupings, region labels), show the helper dict or `Series`.

In [11]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add derived columns expected by multiple scenarios."""
    engineered = df.copy()
    if {"quantity", "unit_price"}.issubset(engineered.columns):
        engineered["computed_total"] = engineered["quantity"] * engineered["unit_price"]
    if {"indate", "outdate"}.issubset(engineered.columns):
        engineered["los_days"] = (engineered["outdate"] - engineered["indate"]).dt.days
    return engineered

# engineered_df = engineer_features(clean_df)

### 6. Data cleaning (20%)

- Detect missing values with `df.isna().sum()` and describe treatment strategies.
- Address negatives or impossible values (drop, cap, absolute) and justify in the decision log.
- Drop low-value columns (`deleted_at`, `estimated_pack_prices`) with a note explaining why.

In [ ]:
from typing import Dict


def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Template for handling missingness, negatives, and redundant columns."""
    cleaned = df.copy()
    summary_missing = cleaned.isna().sum()
    print("Missing values summary (top 10):")
    display(summary_missing.head(10))

    # TODO: implement scenario-appropriate rules below.
    # Example placeholders: remove columns, flag negatives, fill nulls.
    # cleaned = cleaned.drop(columns=["deleted_at", "estimated_pack_prices"], errors="ignore")
    # cleaned.loc[cleaned["quantity"] < 0, "quantity_flag"] = "negative"
    raise NotImplementedError("Replace placeholder rules with your cleaning logic and document decisions.")


def missingness_dict(df: pd.DataFrame) -> Dict[str, float]:
    """
    3) Return dict mapping column -> % missing (0-100).
    """
    total = len(df)
    miss = df.isna().sum().to_dict()
    miss_pct = {col: (cnt / total) * 100 for col, cnt in miss.items()}
    return miss_pct

## TODO change raw_dataf to engineered_df after implementing cleaning and engineering

# QUESTION 3: Missingness dictionary
"This answers the third question on the scenario pdf."
miss_pct = missingness_dict(raw_df)
# Print top 10 most-missing columns
sorted_miss = sorted(miss_pct.items(), key=lambda kv: kv[1], reverse=True)
print("\nQUESTION 3: Column -> % missing (top 15 shown):")
for col, pct in sorted_miss[:15]:
    print(f"  {col}: {pct:.2f}% missing")
# identify top 3 columns with highest missingness
top3_missing = sorted_miss[:3]
print("\nTop 3 columns with highest missingness:")
for col, pct in top3_missing:
    print(f"  {col}: {pct:.2f}%")

#TODO uncomment after implementing cleaning and engineering.
# "This answers the fourth question on the scenario pdf." 
# def high_missing_or_constant_columns(df: pd.DataFrame, missing_pct_threshold: float = 95.0) -> Dict[str, str]:
#     """
#     4) Identify columns with > missing_pct_threshold % missing OR only 1 unique value.

#     Returns a dict mapping column -> reason ('high_missing' or 'single_value').
#     """
#     results = {}
#     miss_pct = missingness_dict(df)
#     for col, pct in miss_pct.items():
#         if pct >= missing_pct_threshold:
#             results[col] = f"high_missing ({pct:.1f}%)"
#         else:
#             # count unique non-null values
#             nunique = df[col].nunique(dropna=True)
#             if nunique <= 1:
#                 results[col] = f"single_value ({nunique} unique non-null values)"
#     return results


# def drop_columns(decision_list: List[str], df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Drop columns listed in decision_list if they exist in df. Return df copy.
#     Documented drop list should include columns like 'dob', 'deleted_at', 'estimated_pack_prices', etc.
#     """
#     cleaned = df.copy()
#     to_drop = [col for col in decision_list if col in cleaned.columns]
#     if to_drop:
#         cleaned = cleaned.drop(columns=to_drop)
#     return cleaned

# # QUESTION 4: Columns with >95% missing or single unique value
# candidates_to_drop = high_missing_or_constant_columns(df, missing_pct_threshold=95.0)
# print("\nQUESTION 4: Columns with >95% missing OR single unique value (col -> reason):")
# if not candidates_to_drop:
#     print("  None detected.")
# else:
#     for c, reason in candidates_to_drop.items():
#         print(f"  {c}: {reason}")

# # Analyst recommended drop list (explicitly required by scenario includes dob, deleted_at, estimated_pack_prices)
# recommended_drops = ["dob", "deleted_at", "estimated_pack_prices"]
# # Add columns marked above (if we decide to drop them)
# for c, reason in candidates_to_drop.items():
#     recommended_drops.append(c)
# # Keep unique recommendations and ensure they exist
# recommended_drops = [c for c in dict.fromkeys(recommended_drops) if c in df.columns]

# print("\nDropping columns (recommended):", recommended_drops)

# df_dropped = drop_columns(recommended_drops, df)

# final_df = clean_data(engineered_df)


QUESTION 3: Column -> % missing (top 15 shown):
  deleted_at: 100.00% missing
  icd10_4: 95.56% missing
  icd10_3: 88.48% missing
  kelas: 81.77% missing
  icd10_2: 69.71% missing
  tarif_price: 66.50% missing
  unit: 60.01% missing
  total_price: 7.84% missing
  item_label: 6.53% missing
  category: 3.75% missing
  tarif_description: 0.96% missing
  benefit_category: 0.96% missing
  chemical_substance: 0.96% missing
  item_type: 0.96% missing
  matched_name: 0.96% missing

Top 3 columns with highest missingness:
  deleted_at: 100.00%
  icd10_4: 95.56%
  icd10_3: 88.48%


NameError: name 'List' is not defined

### 7. Code quality and documentation (5%)

- Refactor repetitive tasks into functions (as scaffolded above).
- Add short comments before non-obvious logic.
- Use markdown headings to organize narrative: context, methods, results, insights.
- Double-check notebook execution order before submission.

## Phase 2: EDA, visualization, and interpretation

### EDA toolkit

Use the helpers below as starting points. Swap in scenario-specific filters or columns.

In [13]:
def describe_numeric(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """Return descriptive statistics for selected numeric columns."""
    stats = df[columns].describe().T
    stats["missing"] = df[columns].isna().sum()
    return stats


def plot_distribution(df: pd.DataFrame, column: str, kind: str = "hist") -> None:
    """Plot a basic distribution for the chosen column."""
    if kind == "hist":
        sns.histplot(df[column].dropna(), kde=False)
    elif kind == "box":
        sns.boxplot(x=df[column].dropna())
    else:
        raise ValueError("kind must be 'hist' or 'box'")
    plt.title(f"Distribution of {column}")
    plt.show()


def plot_grouped_bar(df: pd.DataFrame, index_col: str, value_col: str, top_n: int = 10) -> None:
    """Plot top N categories by aggregated value."""
    summary = (
        df.groupby(index_col)[value_col]
        .sum()
        .sort_values(ascending=False)
        .head(top_n)
    )
    summary.plot(kind="bar")
    plt.ylabel(value_col)
    plt.title(f"Top {top_n} {index_col} by {value_col}")
    plt.xticks(rotation=45, ha="right")
    plt.show()

### Insights narrative

- After generating visuals, write 2-3 paragraphs connecting findings to the business question.
- Call out limitations, data quirks, or assumptions (e.g., synthetic sample, single-claim scope).
- End with actionable recommendations or next steps.

## Scenario playbooks

Each playbook summarizes the official brief. Follow the steps using your cleaned DataFrame. The goal is to show your reasoning path, not to memorize outputs.

### Scenario 1 – Pharmacy Cost Optimization

**Role reminder:** Pharmacy Cost Analyst focusing on medicine items.

**Rubric emphasis:** Negative quantity/price handling, reconstructed totals, scenario-tailored metrics.

**Recommended flow:**
1. Filter to `benefit_category == "medicine"` and `sub_benefit_name == "Obat-Obatan"`.
2. Build raw Python structures from the filtered subset (list of `chemical_substance`, tuple `(kfa_code, generic_name)`, dict `item_label -> chemical_substance`).
3. Convert cost fields to numeric, flag negatives, create `computed_total`, and log decisions.
4. Aggregate cost by item, provider, and province; compute contribution percentages.
5. Plot distributions (histogram/boxplot) and top cost drivers.
6. Summarize anomalies and draft 3-5 recommendations.

**Answering the official questions:**
1. Use raw Python list comprehension over the filtered subset to capture unique `chemical_substance`.
2. Create tuples for immutable identifiers and explain immutability in a markdown cell.
3. Build dict via `{row['item_label']: row['chemical_substance'] for row in subset}` and count entries.
4. After loading data, print `df.shape` and interpret rows vs columns.
5. Cast numeric columns and note parsing issues (e.g., commas, strings).
6. Locate negative values with boolean filters and explain treatment.
7. Compare `computed_total` and `total_price` via a new column or `.assign()`.
8. Justify column drops (e.g., `estimated_pack_prices`).
9. Demonstrate `.loc` for provider/province slices.
10. Conclude with a markdown summary of data quality findings.

### Scenario 2 – Length of Stay & Inpatient Tariff Consistency

**Role reminder:** Hospital Network Performance Analyst.

**Rubric emphasis:** Accurate LOS calculation, tariff handling, date cleaning.

**Recommended flow:**
1. Parse `indate` and `outdate` as datetime and compute `los_days`.
2. Store first 10 LOS values in a list; create tuples `(card_number, indate, outdate)`.
3. Build provider metadata dict with nested attributes.
4. Handle invalid or negative LOS, missing tarif values, and document rationale.
5. Create summary tables for tariff by provider and LOS buckets.
6. Visualize LOS distributions and tariff vs LOS relationships.

**Answering the official questions:**
- Use `.head()` plus `.apply()` to compute the LOS list.
- Explain tuple immutability for patient stay IDs in markdown.
- Construct provider dict via `.drop_duplicates()` then `.set_index().to_dict(orient="index")`.
- Count invalid dates with `df['indate'].isna().sum()` etc.
- Filter rows with negative LOS and describe fixes.
- Convert `tarif_price` to numeric, treat `tarif_description == '-'` as missing.
- Produce average tariff per provider with `groupby`.
- Showcase indexing for `los_days > 5`.
- Document every cleaning step in a dedicated markdown section.

### Scenario 3 – Benefit Category Utilization Across Countries

**Role reminder:** Benefit Design Analyst comparing benefit mix across provinces.

**Rubric emphasis:** Accurate aggregation by benefit dimensions and comparative visuals.

**Recommended flow:**
1. List unique `benefit_category` values and create tuples `(benefit_category, sub_benefit_name)`.
2. Map province to benefit categories using dicts or `defaultdict(list)`.
3. Clean cost columns, drop irrelevant fields, and create aggregated tables.
4. Produce stacked/clustered bars, pies, and heatmaps showing cost distribution.
5. Interpret which provinces or benefit categories drive costs and why.

**Answering the official questions:**
- Use Python list comprehension for unique benefit categories.
- Demonstrate tuple creation and show sample output in markdown.
- Build province → benefits dict via groupby and `.unique()`.
- Report data type checks and missingness for benefit columns.
- Aggregate cost by benefit and province for metrics and visuals.
- Provide narrative on cost shares, outliers, correlations, and recommendations.

### Scenario 4 – ICD-10 Case Mix Profiling

**Role reminder:** Clinical Data Analyst profiling diagnoses and costs.

**Rubric emphasis:** ICD grouping dict, missingness analysis, clinical insights.

**Recommended flow:**
1. Inspect ICD columns for missingness and convert to string.
2. Create list of frequent `icd10_1` values and tuples `(icd10_1, description_1)`.
3. Build a manual ICD grouping dict and apply it safely.
4. Engineer `diagnosis_count` from non-null ICD columns.
5. Aggregate counts and costs by ICD code and group; visualize distributions.
6. Compare costs by diagnosis count and discuss clinical insights with limitations.

**Answering the official questions:**
- Use `.value_counts().head(10)` for top ICD codes.
- Pair codes and descriptions in tuples and explain usage.
- Count missing values and describe handling strategy.
- Plot top ICDs and cost distributions using bar/box charts.
- Discuss anomalies, multi-diagnosis patterns, and future data needs.

### Scenario 5 – Member-Level Exposure & Cost per Member

**Role reminder:** Member Analytics Data Scientist aggregating items to members.

**Rubric emphasis:** Correct member-level aggregation, segmentation by demographics.

**Recommended flow:**
1. Extract unique member identifiers and confirm no duplicates.
2. Build tuples `(card_number, member_name)` and dict `card_number -> sex`.
3. Aggregate to member level: total cost, item count, distinct providers, average unit price.
4. Investigate missing or inconsistent member data, document assumptions (e.g., `dob` as year).
5. Visualize distributions and compare metrics by sex; identify outliers.
6. Produce recommendations for underwriting or care management.

**Answering the official questions:**
- Use `.unique()` for card numbers and `.duplicated()` to check inconsistencies.
- Build per-member summary with `groupby('card_number').agg(...)`.
- Filter and visualize outliers, scatter item count vs cost, boxplots by sex.
- Discuss utilization patterns and actionable insights.

### Scenario 6 – Fraud, Waste, & Abuse Screening

**Role reminder:** Special Investigations Unit analyst flagging anomalies.

**Rubric emphasis:** Strong cleaning, anomaly rules, interpretable watch list.

**Recommended flow:**
1. Identify rows with `quantity <= 0` or `total_price <= 0`; store in a list.
2. Create tuples `(provider_code, benefit_category, description)` for suspicious items.
3. Build dict `provider_code -> suspicious_count`.
4. Define outlier rule for `unit_price` (IQR or percentile) and create `is_suspicious`.
5. Visualize distributions and provider counts; compute cost share of suspicious rows.
6. Summarize watch list and limitations (signals, not proof).

**Answering the official questions:**
- Use boolean indexing to list suspicious rows.
- Count anomalies per provider with `groupby`.
- Plot boxplots and scatterplots highlighting suspicious points.
- Quantify cost share and document reasoning.

### Scenario 7 – Provider Segmentation & Service Mix

**Role reminder:** Provider Network Analyst segmenting providers.

**Rubric emphasis:** Provider-level aggregation, segmentation logic, clear visuals.

**Recommended flow:**
1. List unique providers and confirm metadata consistency.
2. Create tuples `(provider_code, provider_name, province)` and dictionaries summarizing cost by category.
3. Aggregate metrics: total cost, item counts, member counts, cost share by benefit.
4. Define segmentation rules (e.g., >60% medicine cost → medication-heavy).
5. Visualize stacked bars and scatterplots by segment.
6. Interpret segment traits and propose actions.

**Answering the official questions:**
- Compute cost shares, classify providers, and count membership per segment.
- Plot stacked bars and scatter of total cost vs members colored by segment.
- Highlight top cost providers and discuss regional differences.

### Scenario 8 – Claim Cost Prediction & Feature Insight

**Role reminder:** Data analyst supporting predictive modeling.

**Rubric emphasis:** Comprehensive cleaning, feature engineering, insight into predictors.

**Recommended flow:**
1. Inventory numeric columns, data types, and missingness.
2. Drop redundant columns, enforce numeric and datetime types.
3. Engineer features: `claim_total_cost`, `benefit_diversity`, `los_days`, item counts.
4. Validate aggregations (no data loss) and build correlation visuals.
5. Identify key drivers, outliers, and recommend modeling features.
6. Write a readiness narrative summarizing data quality and insights.

**Answering the official questions:**
- Use descriptive stats, missingness dictionary, and column profiling.
- Create aggregation pipelines for claim-level metrics.
- Plot distributions, heatmaps, scatterplots, and interpret correlations.
- Prioritize predictive features and articulate business implications.

## Final checklist before submission

- [ ] Every Phase 1 rubric item has visible evidence (code + explanation).
- [ ] Every Phase 2 rubric item has relevant plots and written interpretation.
- [ ] Scenario-specific questions are answered in code and markdown, referencing figures.
- [ ] Decision log is complete and reflects cleaning choices.
- [ ] Notebook runs top-to-bottom without placeholder errors.
- [ ] Conclusions include limitations and next-step ideas.